In [1]:
from sklearn.datasets import fetch_openml
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

In [2]:
mnist = fetch_openml('mnist_784')

/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [3]:
x, y = mnist['data'], mnist['target']
x=x/255
y=y.astype(np.int8)

In [4]:
x_train, x_test = x[:60000], x[60000:70000]
y_train, y_test = y[:60000], y[60000:70000]



In [5]:
x_train=x_train.values.reshape(784,60000)
x_test=x_test.values.reshape(784,10000)
y_train=y_train.values.reshape(1,60000)
y_test=y_test.values.reshape(1,10000)

In [6]:
y_test.shape

(1, 10000)

In [7]:
x_train.shape

(784, 60000)

In [8]:
Y_train_temp = y_train.reshape(y_train.shape[1],)
Y_train = np.zeros((Y_train_temp.size,10))
Y_train[np.arange(Y_train_temp.size),Y_train_temp] = 1
Y_train = Y_train.T
Y_test_temp = y_test.reshape(y_test.shape[1],)
Y_test = np.zeros((Y_test_temp.size,10))
Y_test[np.arange(Y_test_temp.size),Y_test_temp] = 1
Y_test = Y_test.T

In [9]:
def initialize_parameters_deep(layer_dims):
   parameters = {}
   L = len(layer_dims)
   for l in range(1, L):
          parameters['W' + str(l)] =np.random.randn(layer_dims[l],layer_dims[l-1])*0.01
          parameters['b' + str(l)] =np.zeros((layer_dims[l],1))
   return parameters

In [10]:
def linear_forward(A, W, b):
    Z=np.dot(W,A)+b
    cache=(A,W,b)
    return Z, cache

In [11]:
def sigmoid(z):
   s=1/(1+np.exp(-z))
   cache=(z)
   return s,cache

In [12]:
def relu(z):
  #s=np.maximum(0,z)
  #cache=(z)
  return np.maximum(0,z),z

In [13]:
def softmax(z):
   s = np.exp(z)/np.sum(np.exp(z), axis = 0, keepdims = True)
   #activation_cache = (z)
   return s, z

In [14]:
def linear_activation_forward(A_prev, W, b, activation):

     if activation == "relu":
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache=relu(Z)
     elif activation == "softmax":
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = softmax(Z)
     cache = (linear_cache, activation_cache)

     return A, cache

In [15]:
def L_model_forward(X, parameters):
   caches = []
   A = X
   L = len(parameters) // 2
   for l in range(1,L):
     A_prev=A
     A, cache= linear_activation_forward(A_prev, parameters["W"+str(l)],parameters["b"+str(l)],"relu")
     caches.append(cache)
   AL, cache= linear_activation_forward(A, parameters["W"+str(L)],parameters["b"+str(L)],"softmax")

   caches.append(cache)

   return AL,caches

In [16]:
def compute_cost(AL, Y):
    m = Y.shape[1]
    cost = - np.sum(Y*np.log(AL))/m
    np.squeeze(cost)

    return cost

In [17]:
def linear_backward(dZ, cache):
   A_prev, W, b = cache
   m = A_prev.shape[1]
   dW=(1/m)*(np.dot(dZ,A_prev.T))
   db=(1/m)*np.sum(dZ,axis=1,keepdims=True)
   dA_prev=np.dot(W.T,dZ)
   return dA_prev,dW,db

In [18]:
def softmax_backward(AL, Y):
    dZ = AL- Y
    return dZ

In [19]:
def relu_backward(dA, cache):
    Z = cache
    dZ = np.where(Z > 0, dA, 0)
    return dZ

In [20]:
def linear_activation_backward(Y,AL,dA, cache, activation):
  linear_cache, activation_cache = cache
  if activation == "relu":
    dZ=relu_backward(dA,activation_cache)
    dA_prev,dW,db=linear_backward(dZ,linear_cache)
  elif activation == "softmax":
    dZ=softmax_backward(AL,Y)
    dA_prev,dW,db=linear_backward(dZ,linear_cache)
  return dA_prev, dW, db

In [21]:
def L_model_backward(AL, Y, caches):
  grads = {}
  L = len(caches)
  m = AL.shape[1]
  Y = Y.reshape(AL.shape)
  dAL = -Y/AL
  #dAL=-(np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))
  current_cache = caches[L-1]
  dA_prev_temp, dW_temp, db_temp =linear_activation_backward(Y,AL,dAL,current_cache,"softmax")
  grads["dA" + str(L-1)] = dA_prev_temp
  grads["dW" + str(L)] = dW_temp
  grads["db" + str(L)] = db_temp
  for l in reversed(range(L-1)):
    current_cache = caches[l]
    dA_prev_temp, dW_temp, db_temp =linear_activation_backward(Y,AL,grads["dA"+str(l+1)],current_cache,"relu")
    grads["dA" + str(l)] = dA_prev_temp
    grads["dW" + str(l+1)] = dW_temp
    grads["db" + str(l+1)] = db_temp
  return grads

In [22]:
def update_parameters(params, grads, learning_rate):
  parameters = params.copy()
  L = len(parameters) // 2
  for l in range(L):
        parameters["W" + str(l+1)] =parameters["W" + str(l+1)]-learning_rate*grads["dW"+str(l+1)]
        parameters["b" + str(l+1)] =parameters["b" + str(l+1)]-learning_rate*grads["db"+str(l+1)]

        # YOUR CODE ENDS HERE
  return parameters

In [23]:
def get_predictions(Y_hat):
    return np.argmax(Y_hat,0)

def get_accuracy(predictions,Y):
    predictions = predictions.reshape(1,predictions.shape[0])
    #print(predictions.shape)
    ans = 0
    for i in range(Y.shape[1]) :
        predict = predictions[0,i]
        if Y[predict,i]==1 :
            ans+=1
   # print(ans)
    return str((ans/Y.shape[1])*100) + '%'

In [24]:
layers_dims = [784, 30, 20, 10]

In [25]:

def L_layer_model(X, Y, layers_dims, learning_rate = 0.0075, num_iterations = 3000,print_cost=False):
  #np.random.seed(1)
  grads = {}
  costs = []
  m = X.shape[1]
  #print(layers_dims)
  cost = 2.5
  parameters=initialize_parameters_deep(layers_dims)
  for i in range(0, num_iterations):
    AL, caches=L_model_forward(X, parameters)
    grads=L_model_backward(AL, Y, caches)
    parameters=update_parameters(parameters, grads, learning_rate)
    cost = compute_cost(AL,Y)
    Y_predict = np.zeros(AL.shape)
    Y_predict[np.argmax(AL, axis = 0), np.arange(AL.shape[1])] = 1
    if print_cost and i % 100 == 0 or i == num_iterations - 1:
            print("Cost after iteration {}: {}".format(i, np.squeeze(cost)))
            #print("train accuracy: {} %".format(100 - np.mean(np.abs(Y_predict - Y)) * 100))
            print("accuracy : " , get_accuracy(get_predictions(AL),Y))

    if i % 100 == 0 or i == num_iterations:
            costs.append(cost)

  return parameters,costs, Y_predict

In [28]:
parameters, costs, Y_predict =L_layer_model(x_train,Y_train, layers_dims, 0.2, 2500, True)

Cost after iteration 0: 2.30258520512949
accuracy :  9.411666666666667%
Cost after iteration 100: 2.301182379512185
accuracy :  11.236666666666666%
Cost after iteration 200: 2.301159171326631
accuracy :  11.236666666666666%
Cost after iteration 300: 2.301158597961851
accuracy :  11.236666666666666%
Cost after iteration 400: 2.301158439874312
accuracy :  11.236666666666666%
Cost after iteration 500: 2.3011582726101962
accuracy :  11.236666666666666%
Cost after iteration 600: 2.3011580982959186
accuracy :  11.236666666666666%
Cost after iteration 700: 2.301157936278211
accuracy :  11.236666666666666%
Cost after iteration 800: 2.3011577973837585
accuracy :  11.236666666666666%
Cost after iteration 900: 2.3011576628548336
accuracy :  11.236666666666666%
Cost after iteration 1000: 2.301157533160655
accuracy :  11.236666666666666%
Cost after iteration 1100: 2.3011574106133383
accuracy :  11.236666666666666%
Cost after iteration 1200: 2.3011573005518406
accuracy :  11.236666666666666%
Cost af

KeyboardInterrupt: ignored